## Crawler

In [2]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import re

#각 크롤링 결과 저장하기 위한 리스트 선언 
title_text=[]
link_text=[]
source_text=[]
date_text=[]
contents_text=[]
result={}

#엑셀로 저장하기 위한 변수
RESULT_PATH ='C:/Users/User/Desktop/python study/beautifulSoup_ws/crawling_result/'  #결과 저장할 경로
now = datetime.now() #파일이름 현 시간으로 저장하기

#날짜 정제화 함수
def date_cleansing(test):
    try:
        #지난 뉴스
        #머니투데이  10면1단  2018.11.05.  네이버뉴스   보내기  
        pattern = '\d+.(\d+).(\d+).'  #정규표현식 
        r = re.compile(pattern)
        match = r.search(test).group(0)  # 2018.11.05.
        date_text.append(match)
        
    except AttributeError:
        #최근 뉴스
        #이데일리  1시간 전  네이버뉴스   보내기  
        pattern = '\w* (\d\w*)'     #정규표현식 
        r = re.compile(pattern)
        match = r.search(test).group(1)
        #print(match)
        date_text.append(match)


#내용 정제화 함수 
def contents_cleansing(contents):
    first_cleansing_contents = re.sub('<dl>.*?</a> </div> </dd> <dd>', '', 
                                      str(contents)).strip()  #앞에 필요없는 부분 제거
    second_cleansing_contents = re.sub('<ul class="relation_lst">.*?</dd>', '', 
                                       first_cleansing_contents).strip()#뒤에 필요없는 부분 제거 (새끼 기사)
    third_cleansing_contents = re.sub('<.+?>', '', second_cleansing_contents).strip()
    contents_text.append(third_cleansing_contents)
    #print(contents_text)

## DataFrame

In [7]:
from datetime import date, timedelta

def crawler(maxpage,query,sort,s_date,e_date):
    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1  
    maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지
    while page <= maxpage_t:
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        
        response = requests.get(url)
        html = response.text
 
        #뷰티풀소프의 인자값 지정
        soup = BeautifulSoup(html, 'html.parser')
        #<a>태그에서 제목과 링크주소 추출
        atags = soup.select('.news_tit')
        for atag in atags:
            title_text.append(atag.text)     #제목
            link_text.append(atag['href'])   #링크주소
        #신문사 추출
        source_lists = soup.select('.info_group > .press')
        for source_list in source_lists:
            source_text.append(source_list.text)    #신문사
        #날짜 추출 
        date_lists = soup.select('.info_group > span.info')
        for date_list in date_lists:
            # 1면 3단 같은 위치 제거
            if date_list.text.find("면") == -1:
                date_text.append(date_list.text)
        
        #본문요약본
        contents_lists = soup.select('.news_dsc')
        for contents_list in contents_lists:
            contents_cleansing(contents_list) #본문요약 정제화
        

        #모든 리스트 딕셔너리형태로 저장
        result= {"date" : date_text , "title":title_text ,  "source" : source_text ,"contents": contents_text ,"link":link_text }  
        # print(page)
        df = pd.DataFrame(result)
        page += 10
    # 새로 만들 파일이름 지정
    # df = pd.read_csv('/content/news_crawler.csv')
    df = df[['date','title']]
    return df

#### 검색어 입력 ####
###################
#### Start Date ####
s_date = '2022.08.01'
query = '포스코케미칼'
days = 30
maxpage =2
####################
#####################

string = s_date.split('.')
start_date = date(int(string[0]), int(string[1]), int(string[2]))

### days : + 며칠인지
end_dat = start_date + timedelta(days=days)

while start_date != end_dat:
  df = crawler('1',query,'1',s_date,s_date)
  df = df[['date','title']]
  start_date = start_date + timedelta(days=1)
  string = str(start_date).split('-')
  s_date = ".".join(string)

df

,date,title
0,2022.08.01.,[아주경제 오늘의 뉴스 종합] 한 달 앞으로 다가온 추석...폭염에 장마까지 여...
1,2022.08.01.,"포스코 지주사 포항이전 범대위 ""최정우 회장 포항시민들에게 상생의지 밝혀야..."
2,2022.08.01.,"코스피, 외인·개인 매수에 2,450선 지켜…외인 순매수 상위 종목은 삼성SDI·..."
3,2022.08.01.,폴란드 이차전지 양극재 시장동향
4,2022.08.01.,美 원자재 써야 전기차 보조금…中 배제 법안에 韓배터리 '방긋'
...,...,...
742,2022.08.30.,TBC)상반기 지역 상장법인 매출 53.80% ↑순이익 54.48% ↑
743,2022.08.30.,"개인, 공매도 몰린 종목 ‘사자’…외인·기관은 ‘태조이방원’ [증시프리즘..."
744,2022.08.30.,"LG엔솔, 日 혼다와 美 배터리합작공장 추진. 전문가들 의견은?"
745,2022.08.30.,"[증시 돋보기] 외국인 현대차 기아 순매수, 삼성전자 순매도 1위"


In [8]:
df['date'].unique()

array(['2022.08.01.', '2022.08.02.', '2022.08.03.', '2022.08.04.',
       '2022.08.05.', '2022.08.06.', '2022.08.07.', '2022.08.08.',
       '2022.08.09.', '2022.08.10.', '2022.08.11.', '2022.08.12.',
       '2022.08.13.', '2022.08.14.', '2022.08.15.', '2022.08.16.',
       '2022.08.17.', '2022.08.18.', '2022.08.19.', '2022.08.20.',
       '2022.08.21.', '2022.08.22.', '2022.08.23.', '2022.08.24.',
       '2022.08.25.', '2022.08.26.', '2022.08.27.', '2022.08.28.',
       '2022.08.29.', '2022.08.30.', '2022.08.31.', '2022.09.01.',
       '2022.09.02.', '2022.09.03.', '2022.09.04.', '2022.09.05.',
       '2022.09.06.', '2022.09.07.', '2022.09.08.', '2022.09.09.',
       '2022.09.10.', '2022.09.11.', '2022.09.12.', '2022.09.13.',
       '2022.09.14.', '2022.09.15.', '2022.09.16.', '2022.09.17.',
       '2022.09.18.', '2022.09.19.'], dtype=object)

## Data Preprocessing

## Save DataFrame

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
df.to_csv("30posco_chemical_news.csv")